In [1]:
import pandas as pd
import json
import sklearn.svm as svm
import numpy as np
from sklearn.utils import shuffle

In [2]:
## Change the name to get your path
user = 'Andrew'
file1 = 'AG01-01.csv'
file2 = 'GTH01-01.csv'

## Test lag function

In [3]:
from script import *

In [32]:
def get_lags(data, lags):
    prev_data = data[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
    for i in range(lags):
        copy = prev_data.copy(deep=True)
        print(copy.index)
        copy.iloc[-1,:] = copy.iloc[0,:]
        #print(copy)
        copy.index = copy.index + 1  # shifting index
        copy.sort_index(inplace=True)
        copy.columns = 'last.' + str(i+1) + "." + copy.columns
        print(copy)
        prev_data = copy
        data = pd.concat([data, copy], axis = 1)
        data = data.drop(data.index[len(data)-1])
    return data

In [4]:
struct = load_struct()
observed_data = get_all_subjects(user, struct)

C:\Users\Andrew\Documents\CalPoly\DataCapstone\KineseItUp\script.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  observedData['index'] = range(len(observedData))


In [5]:
print(observed_data)

      Unnamed: 0   mean.vm     sd.vm   mean.ang     sd.ang      p625  \
0       368941.0  1.025459  0.052283 -13.663296   3.477743  0.072835   
1       368942.0  1.019744  0.028089 -16.673945   2.361258  0.059919   
2       368943.0  1.027147  0.026020  -6.451596   5.413729  0.216824   
3       368944.0  1.049957  0.119535 -21.404693  23.206850  0.299554   
4       368945.0  1.028554  0.049378 -69.353741   1.962348  0.130182   
5       368946.0  1.043464  0.095178 -57.000094  11.747111  0.307349   
6       368947.0  1.027036  0.065807 -32.179828   8.090852  0.188766   
7       368948.0  1.029727  0.136367 -32.955579   8.908046  0.222211   
8       368949.0  1.003435  0.056664 -47.568056   6.244429  0.235925   
9       368950.0  1.020198  0.070674 -54.486552   4.062847  0.479812   
10      368951.0  1.003209  0.031647 -52.630436   2.084940  0.443848   
11      368952.0  1.008673  0.035287 -54.546043   2.104136  0.316200   
12      368953.0  1.013144  0.008689 -56.388660   0.873721  0.23

## Rest of file

In [ ]:
def load_Data(user, filename):
   data = json.load(open('dir.json'))
   dir = data[user] + filename
#    data = pd.read_csv(dir, header=2)
   return dir

In [4]:
data = pd.read_csv(load_Data(user, file1), header=0)
data['start.time'] = pd.to_datetime(data['start.time'])
## selecting the times when the ground truth was observed
observedData = data.copy(deep=True)
observedData = observedData[(observedData['start.time'] >= '2017-10-03 16:44:00') & (observedData['start.time'] <= '2017-10-03 18:44:00')]
observedData.loc[:,'index'] = range(len(observedData))

In [5]:
data.head()

,index,mean.vm,sd.vm,mean.ang,sd.ang,p625,dfreq,ratio.df,start.time
0,1,1.008720,0.005388,14.405906,0.178485,0.073094,6.153846,0.079059,2017-10-02 10:15:00
1,2,1.008625,0.006601,14.444647,0.234531,0.074642,4.102564,0.174183,2017-10-02 10:15:00
2,3,1.018935,0.023567,12.862005,2.996361,0.309095,1.025641,0.158892,2017-10-02 10:15:00
3,4,1.019761,0.066683,-1.952166,12.679718,0.114479,2.051282,0.101853,2017-10-02 10:15:00
4,5,1.043682,0.056193,-16.001678,5.775647,0.217691,1.025641,0.149609,2017-10-02 10:15:00


In [7]:
gt = pd.read_csv(load_Data(user, file2), header=0)
gt['index'] = gt['time']
# gt.head()

In [8]:
observedData = pd.merge(observedData,gt[['index','coding']])

In [9]:
observedData = shuffle(observedData)
pivot = round(len(observedData) * 0.7)
train = observedData[:pivot]
test = observedData[pivot:]
## this might be an issue since there is ordinality and that makes things weird.  The later parts of this trial
## were more likely to be sedentary.
X = train[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
y = train['coding']

testX = test[['mean.vm','sd.vm','mean.ang','sd.ang','p625','dfreq','ratio.df']]
testY = test['coding']

In [10]:
clf = svm.SVC(decision_function_shape='ovo')
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [11]:
clf.predict([[1,.0004,1,1,1,9,1],[.5,.002,.1,3,1,.5,1]])

array(['non-sedentary', 'sedentary'], dtype=object)

In [12]:
## get accuracy of the svm
clf.score(testX,testY)

0.74242424242424243

In [13]:
linearKernel = svm.LinearSVC()
linearKernel.fit(X, y)
linearKernel.score(testX, testY)

0.65013774104683197

In [17]:
## what if we tried a neural net for fun?
from sklearn.neural_network import MLPClassifier

In [18]:
n_net = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(2000, 1000), random_state=1)
n_net.fit(X,y)
## test accuracy of neural net
np.sum(n_net.predict(X) == y) / len(X)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(2000, 1000), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

## Run GridSearch for parameter tuning on Neural Network

In [14]:
from sklearn.model_selection import GridSearchCV

In [18]:
parameters = {'activation': ['logistic', 'relu', 'tanh'], 
              'hidden_layer_sizes': [(2000, 1000), (1000, 500, 250, 125), (500, 400, 300, 200, 100, 50),
                                     (200, 200, 150, 150, 100, 100, 50, 50, 25, 25, 10, 10)]}
model = MLPClassifier()

In [19]:
nn_grid = GridSearchCV(model, parameters)

In [20]:
nn_grid.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'activation': ['logistic', 'relu', 'tanh'], 'hidden_layer_sizes': [(2000, 1000), (1000, 500, 250, 125), (500, 400, 300, 200, 100, 50), (200, 200, 150, 150, 100, 100, 50, 50, 25, 25, 10, 10)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [21]:
nn_grid.best_params_

{'activation': 'logistic', 'hidden_layer_sizes': (1000, 500, 250, 125)}

In [22]:
nn_grid.best_score_

0.67066692897894942

## Run GridSearch for parameter tuning on SVM

In [23]:
clf = svm.SVC(cache_size=7000)
parameter_grid = {'C': [0.001, 0.01, 0.1, 1, 10], 
                  'kernel': ['linear', 'rbf', 'sigmoid'], 
                  'gamma': [0.001, 0.01, 0.1, 1]}
svm_search = GridSearchCV(clf, parameter_grid)

In [24]:
svm_search.fit(X, y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=7000, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10], 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [0.001, 0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [26]:
print(svm_search.best_params_)
print(svm_search.best_score_)

{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
0.717883139878
